In [79]:
import pandas as pd 
import numpy as np 
import plotly.express as px

In [80]:
ses = pd.read_csv("../Datasets/PROCESSED/ses2.csv")
ses.head()

,İLÇE ADI,MAHALLE ADI,SES,SES SKORU
0,ADALAR,BURGAZADA,B,62.5
1,ADALAR,HEYBELIADA,B,62.5
2,ADALAR,KINALIADA,C,37.5
3,ADALAR,MADEN,B,62.5
4,ADALAR,NIZAM,B+,75.0


In [81]:
brew = pd.read_csv("../Datasets/PROCESSED/brewstation.csv")
brew['mahalle'] = brew['mahalle'].str.replace(" ", "")
brew = pd.merge(brew, ses, left_on=['ilce','mahalle'], right_on=["İLÇE ADI", 'MAHALLE ADI'], how="left")
brew.head(3)

,sehir,acik_adres,ilce,mahalle,nufus_yogunlugu,nufus,yuzolcumu,maliyet,İLÇE ADI,MAHALLE ADI,SES,SES SKORU
0,İSTANBUL,"KOCATEPE MAH PAŞA CAD NO:3, FORUM ISTANBUL AVM...",BAYRAMPASA,KOCATEPE,12394.0,21448,1.7,300000,BAYRAMPASA,KOCATEPE,C,37.5
1,İSTANBUL,MAHMUTBEY MAHALLESİ TAŞOCAĞI YOLU CAD. A BLOK ...,BAGCILAR,MAHMUTBEY,5225.0,24596,4.7,315000,BAGCILAR,MAHMUTBEY,C,37.5
2,İSTANBUL,"MASLAK MAHALLESI, AHİ EVRAN CAD NO:6/A 42 SHOP...",SARIYER,MASLAK,1328.0,8091,6.1,819000,SARIYER,MASLAK,A,87.5


In [82]:
brew.drop(['acik_adres','yuzolcumu', 'sehir','SES','MAHALLE ADI','İLÇE ADI'], axis=1, inplace=True)

In [83]:
database = pd.read_csv("../Datasets/PROCESSED/database.csv")
database.head()

,İLÇE ADI,MAHALLE ADI,SES SKORU,nufus,nufus_yogunlugu,maliyet,agirlikli_puan
0,KADIKOY,CAFERAGA,100.0,23977,19324.0,567000,0.658805
1,BAKIRKOY,ATAKOY7-8-9-10.KISIM,100.0,24444,11918.0,525000,0.656336
2,SISLI,TESVIKIYE,100.0,11522,25431.0,543000,0.649799
3,BEYKOZ,ACARLAR,100.0,7319,1346.0,363000,0.643938
4,KADIKOY,SUADIYE,100.0,22484,15157.0,567000,0.640568


In [84]:
from sklearn.preprocessing import MinMaxScaler

nufus_yog_scaler = MinMaxScaler()
database["nufus_yogunlugu_agirlik"] = nufus_yog_scaler.fit_transform(database[["nufus_yogunlugu"]])

nufus_scaler = MinMaxScaler()
database["nufus_agirlik"] = nufus_scaler.fit_transform(database[["nufus"]])

maaliyet_scaler = MinMaxScaler()
database["maliyet_agirlik"] = 1 - maaliyet_scaler.fit_transform(database[["maliyet"]])

ses_scaler = MinMaxScaler()
database["ses_agirlik"] = ses_scaler.fit_transform(database[["SES SKORU"]])

database["agirlikli_puan"] = (database["nufus_yogunlugu_agirlik"] * 0.2
                            + database["nufus_agirlik"] * 0.1
                            + database["maliyet_agirlik"] * 0.3
                            + database["ses_agirlik"] * 0.4)

database = database.sort_values('agirlikli_puan', ascending=False)

In [85]:
database.head()

,İLÇE ADI,MAHALLE ADI,SES SKORU,nufus,nufus_yogunlugu,maliyet,agirlikli_puan,nufus_yogunlugu_agirlik,nufus_agirlik,maliyet_agirlik,ses_agirlik
0,KADIKOY,CAFERAGA,100.0,23977,19324.0,567000,0.658805,0.311412,0.770911,0.398104,1.0
1,BAKIRKOY,ATAKOY7-8-9-10.KISIM,100.0,24444,11918.0,525000,0.656336,0.192032,0.785935,0.464455,1.0
2,SISLI,TESVIKIYE,100.0,11522,25431.0,543000,0.649799,0.409853,0.370223,0.436019,1.0
3,BEYKOZ,ACARLAR,100.0,7319,1346.0,363000,0.643938,0.021618,0.235008,0.720379,1.0
4,KADIKOY,SUADIYE,100.0,22484,15157.0,567000,0.640568,0.244243,0.722880,0.398104,1.0


In [86]:
brew.head(3)

,ilce,mahalle,nufus_yogunlugu,nufus,maliyet,SES SKORU
0,BAYRAMPASA,KOCATEPE,12394.0,21448,300000,37.5
1,BAGCILAR,MAHMUTBEY,5225.0,24596,315000,37.5
2,SARIYER,MASLAK,1328.0,8091,819000,87.5


In [87]:
brew["nufus_yogunlugu_agirlik"] = nufus_yog_scaler.transform(brew[["nufus_yogunlugu"]])

brew["nufus_agirlik"] = nufus_scaler.transform(brew[["nufus"]])

brew["maliyet_agirlik"] = 1 - maaliyet_scaler.transform(brew[["maliyet"]])

brew["ses_agirlik"] = ses_scaler.transform(brew[["SES SKORU"]])

brew["agirlikli_puan"] = (brew["nufus_yogunlugu_agirlik"] * 0.2
                            + brew["nufus_agirlik"] * 0.1
                            + brew["maliyet_agirlik"] * 0.3
                            + brew["ses_agirlik"] * 0.4)

brew = brew.sort_values('agirlikli_puan', ascending=False)

In [88]:
brew.head()

,ilce,mahalle,nufus_yogunlugu,nufus,maliyet,SES SKORU,nufus_yogunlugu_agirlik,nufus_agirlik,maliyet_agirlik,ses_agirlik,agirlikli_puan
247,KUCUKCEKMECE,ATAKENT,10949.0,93229,315000,75.0,0.176412,2.998810,0.796209,0.714286,0.859740
28,KUCUKCEKMECE,ATAKENT,10949.0,93229,315000,75.0,0.176412,2.998810,0.796209,0.714286,0.859740
52,BEYLIKDUZU,ADNANKAHVECI,18865.0,86584,291000,62.5,0.304013,2.785034,0.834123,0.571429,0.818114
54,BEYLIKDUZU,ADNANKAHVECI,18865.0,86584,291000,62.5,0.304013,2.785034,0.834123,0.571429,0.818114
69,BAHCELIEVLER,BAHCELIEVLER,19452.0,61832,264000,75.0,0.313475,1.988740,0.876777,0.714286,0.810317


In [89]:
brew['agirlikli_puan'].max(), brew['agirlikli_puan'].min()

(0.8597402484891683, 0.1727336921708834)

In [90]:
brew.sort_values('agirlikli_puan', ascending=True).head(10)

,ilce,mahalle,nufus_yogunlugu,nufus,maliyet,SES SKORU,nufus_yogunlugu_agirlik,nufus_agirlik,maliyet_agirlik,ses_agirlik,agirlikli_puan
248,BEYOGLU,KEMANKESKARAMUSTAFAPASA,905.0,128,582000,25.0,0.014509,0.003667,0.374408,0.142857,0.172734
83,BEYOGLU,KEMANKESKARAMUSTAFAPASA,905.0,128,582000,25.0,0.014509,0.003667,0.374408,0.142857,0.172734
81,BEYOGLU,KEMANKESKARAMUSTAFAPASA,905.0,128,582000,25.0,0.014509,0.003667,0.374408,0.142857,0.172734
134,BEYOGLU,SEHITMUHTAR,15608.0,1493,582000,25.0,0.251512,0.047581,0.374408,0.142857,0.224526
260,SARIYER,AYAZAGA,2264.0,35317,819000,37.5,0.036415,1.135729,0.000000,0.285714,0.235142
261,SARIYER,AYAZAGA,2264.0,35317,819000,37.5,0.036415,1.135729,0.000000,0.285714,0.235142
232,SARIYER,AYAZAGA,2264.0,35317,819000,37.5,0.036415,1.135729,0.000000,0.285714,0.235142
263,SARIYER,AYAZAGA,2264.0,35317,819000,37.5,0.036415,1.135729,0.000000,0.285714,0.235142
262,SARIYER,AYAZAGA,2264.0,35317,819000,37.5,0.036415,1.135729,0.000000,0.285714,0.235142
224,BEYKOZ,RUZGARLIBAHCE,2331.0,8011,363000,12.5,0.037495,0.257271,0.720379,0.000000,0.249340


In [91]:
fig = px.box(brew, x="agirlikli_puan", orientation='h', title="Brewstation'ların ilçelere göre ağırlıklı puanları")
fig.show()

In [92]:
brew.groupby(['ilce']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(10)

,ilce,counts
27,SISLI,22
25,SARIYER,22
18,KADIKOY,22
1,ATASEHIR,18
8,BESIKTAS,18
5,BAKIRKOY,14
32,USKUDAR,12
31,UMRANIYE,12
30,TUZLA,10
22,MALTEPE,10


In [93]:
fig = px.bar(brew.groupby(['ilce']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(20),
                y='counts', x='ilce', title="Brewstation'ların ilçelere göre sayıları", color='ilce')
fig.update_layout(showlegend=False, width=1000, height=400)
fig.show()

In [94]:
brew.fillna(0, inplace=True)

In [95]:
fig = px.scatter(brew, x='agirlikli_puan', y='nufus_yogunlugu', size='SES SKORU', color='ilce')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                  width=1400, height=450)
fig.show()

In [96]:
brewstation = brew.groupby(['ilce','mahalle']).size().reset_index(name="brew_sube_sayisi")

In [97]:
brewstation.head()

,ilce,mahalle,brew_sube_sayisi
0,ARNAVUTKOY,TAYAKADIN,4
1,ATASEHIR,ATATURK,5
2,ATASEHIR,BARBAROS,5
3,ATASEHIR,ICERENKOY,2
4,ATASEHIR,KUCUKBAKKALKOY,5


In [98]:
database.head()

,İLÇE ADI,MAHALLE ADI,SES SKORU,nufus,nufus_yogunlugu,maliyet,agirlikli_puan,nufus_yogunlugu_agirlik,nufus_agirlik,maliyet_agirlik,ses_agirlik
0,KADIKOY,CAFERAGA,100.0,23977,19324.0,567000,0.658805,0.311412,0.770911,0.398104,1.0
1,BAKIRKOY,ATAKOY7-8-9-10.KISIM,100.0,24444,11918.0,525000,0.656336,0.192032,0.785935,0.464455,1.0
2,SISLI,TESVIKIYE,100.0,11522,25431.0,543000,0.649799,0.409853,0.370223,0.436019,1.0
3,BEYKOZ,ACARLAR,100.0,7319,1346.0,363000,0.643938,0.021618,0.235008,0.720379,1.0
4,KADIKOY,SUADIYE,100.0,22484,15157.0,567000,0.640568,0.244243,0.722880,0.398104,1.0


In [99]:
database = pd.merge(database, brewstation, left_on=['İLÇE ADI','MAHALLE ADI'], right_on=['ilce','mahalle'], how="left")

In [100]:
database.drop(['ilce','mahalle'], axis=1, inplace=True)

In [101]:
database['brew_sube_sayisi'].fillna(0, inplace=True)

In [102]:
database.to_csv("../Datasets/PROCESSED/database.csv", index=False)